# Water Demand Prediction for Ohrid, North Macedonia

## Research Framework Demo

This notebook demonstrates the comprehensive water demand prediction framework specifically designed for Ohrid, incorporating:

- **Regional Characteristics**: Tourism patterns, Mediterranean climate, cultural factors
- **Traditional Time Series**: ARIMA, SARIMA, Exponential Smoothing
- **Machine Learning**: Random Forest, XGBoost, LightGBM
- **Deep Learning**: Neural Networks, LSTM
- **Hybrid Approaches**: Ensemble methods
- **Comprehensive Evaluation**: Multiple metrics including peak demand analysis

In [ ]:
# Import required libraries
import sys
import os
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Import our custom modules
from data_collectors.ohrid_synthetic_generator import OhridWaterDemandGenerator
from models.ohrid_predictor import OhridWaterDemandPredictor

# Set display options
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8')

print("✅ Libraries imported successfully")
print("📍 Research focus: Ohrid, North Macedonia")
print("🎯 Objective: Water demand prediction using multiple approaches")

## 1. Generate Realistic Synthetic Data for Ohrid

We'll generate synthetic water demand data that captures Ohrid's unique characteristics:

- **Tourism Seasonality**: UNESCO World Heritage site with peak summer tourism
- **Cultural Events**: Ohrid Summer Festival, Orthodox holidays
- **Climate**: Mediterranean with continental influence
- **Infrastructure**: Lake Ohrid as water source, mixed-age distribution network

In [ ]:
# Initialize the Ohrid-specific data generator
generator = OhridWaterDemandGenerator(config_path='../config/ohrid_config.yaml')

# Generate 3 years of hourly data (2021-2023)
print("🔄 Generating synthetic data for Ohrid (2021-2023)...")
synthetic_data = generator.generate_synthetic_data(
    start_date="2021-01-01",
    end_date="2023-12-31",
    frequency="1h"
)

print(f"✅ Generated {len(synthetic_data):,} hourly observations")
print(f"📊 Data shape: {synthetic_data.shape}")
print(f"📅 Date range: {synthetic_data['timestamp'].min()} to {synthetic_data['timestamp'].max()}")

# Display basic statistics
print("\n📈 Water Demand Statistics:")
print(f"Average demand: {synthetic_data['water_demand_m3_per_hour'].mean():.2f} m³/hour")
print(f"Peak demand: {synthetic_data['water_demand_m3_per_hour'].max():.2f} m³/hour")
print(f"Minimum demand: {synthetic_data['water_demand_m3_per_hour'].min():.2f} m³/hour")
print(f"Standard deviation: {synthetic_data['water_demand_m3_per_hour'].std():.2f} m³/hour")

In [ ]:
# Preview the generated data
print("🔍 Sample of generated data:")
display(synthetic_data.head(10))

print("\n📋 Data columns:")
for i, col in enumerate(synthetic_data.columns, 1):
    print(f"{i:2d}. {col}")

## 2. Exploratory Data Analysis

Let's explore the patterns in our synthetic data to understand Ohrid's water demand characteristics.

In [ ]:
# Time series plot of water demand
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        'Water Demand Over Time',
        'Seasonal Patterns',
        'Daily Patterns',
        'Tourism Impact'
    ],
    specs=[[{"colspan": 2}, None],
           [{}, {}]]
)

# Overall time series (sample every 24 hours for clarity)
sample_data = synthetic_data[::24].copy()  # Daily samples
fig.add_trace(
    go.Scatter(
        x=sample_data['timestamp'],
        y=sample_data['water_demand_m3_per_hour'],
        mode='lines',
        name='Daily Demand',
        line=dict(color='blue', width=1)
    ),
    row=1, col=1
)

# Monthly averages
monthly_avg = synthetic_data.groupby('month')['water_demand_m3_per_hour'].mean()
fig.add_trace(
    go.Bar(
        x=monthly_avg.index,
        y=monthly_avg.values,
        name='Monthly Average',
        marker_color='lightblue'
    ),
    row=2, col=1
)

# Hourly patterns
hourly_avg = synthetic_data.groupby('hour')['water_demand_m3_per_hour'].mean()
fig.add_trace(
    go.Scatter(
        x=hourly_avg.index,
        y=hourly_avg.values,
        mode='lines+markers',
        name='Hourly Pattern',
        line=dict(color='red')
    ),
    row=2, col=2
)

fig.update_layout(
    height=600,
    title_text="Ohrid Water Demand Analysis",
    showlegend=False
)

fig.update_xaxes(title_text="Month", row=2, col=1)
fig.update_xaxes(title_text="Hour of Day", row=2, col=2)
fig.update_yaxes(title_text="Demand (m³/hour)", row=1, col=1)
fig.update_yaxes(title_text="Demand (m³/hour)", row=2, col=1)
fig.update_yaxes(title_text="Demand (m³/hour)", row=2, col=2)

fig.show()

In [ ]:
# Tourism impact analysis
tourism_analysis = synthetic_data.groupby(['month', 'is_tourist_season']).agg({
    'water_demand_m3_per_hour': 'mean',
    'tourists_estimated': 'mean'
}).reset_index()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Water demand by tourist season
sns.boxplot(data=synthetic_data, x='month', y='water_demand_m3_per_hour', 
            hue='is_tourist_season', ax=ax1)
ax1.set_title('Water Demand by Month and Tourist Season')
ax1.set_xlabel('Month')
ax1.set_ylabel('Water Demand (m³/hour)')
ax1.legend(title='Tourist Season')

# Tourist numbers throughout the year
monthly_tourists = synthetic_data.groupby('month')['tourists_estimated'].mean()
ax2.plot(monthly_tourists.index, monthly_tourists.values, marker='o', linewidth=2, markersize=8)
ax2.set_title('Average Tourist Numbers by Month')
ax2.set_xlabel('Month')
ax2.set_ylabel('Estimated Tourists')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("🏖️ Tourism Impact Insights:")
print(f"Peak tourist month: {monthly_tourists.idxmax()} (avg: {monthly_tourists.max():.0f} tourists)")
print(f"Off-season month: {monthly_tourists.idxmin()} (avg: {monthly_tourists.min():.0f} tourists)")
tourist_demand_increase = (
    synthetic_data[synthetic_data['is_tourist_season']]['water_demand_m3_per_hour'].mean() /
    synthetic_data[~synthetic_data['is_tourist_season']]['water_demand_m3_per_hour'].mean() - 1
) * 100
print(f"Demand increase during tourist season: +{tourist_demand_increase:.1f}%")

## 3. Data Preparation and Feature Engineering

Now we'll prepare the data for modeling using our comprehensive feature engineering pipeline.

In [ ]:
# Initialize the predictor
predictor = OhridWaterDemandPredictor(config_path='../config/ohrid_config.yaml')

# Load and prepare data
print("🔧 Applying feature engineering...")
df = predictor.load_data('../data/raw/ohrid_synthetic_water_demand.csv') if os.path.exists('../data/raw/ohrid_synthetic_water_demand.csv') else synthetic_data.copy()

# Prepare data for modeling
X_train, X_val, X_test, y_train, y_val, y_test, feature_cols = predictor.prepare_data_for_modeling(df)

print(f"📊 Data split summary:")
print(f"   Training samples: {len(X_train):,}")
print(f"   Validation samples: {len(X_val):,}")
print(f"   Test samples: {len(X_test):,}")
print(f"   Features: {len(feature_cols)}")

print(f"\n🎯 Target variable statistics:")
print(f"   Training mean: {y_train.mean():.2f} m³/hour")
print(f"   Training std: {y_train.std():.2f} m³/hour")
print(f"   Training range: {y_train.min():.2f} - {y_train.max():.2f} m³/hour")

In [ ]:
# Display engineered features
print("🛠️ Engineered Features:")
for i, feature in enumerate(feature_cols, 1):
    print(f"{i:2d}. {feature}")

# Check for missing values
missing_values = X_train.isnull().sum()
if missing_values.sum() > 0:
    print("\n⚠️ Missing values detected:")
    print(missing_values[missing_values > 0])
else:
    print("\n✅ No missing values in training data")

## 4. Model Training

We'll train multiple types of models:
1. Traditional time series models (ARIMA, ETS)
2. Machine learning models (Random Forest, XGBoost, LightGBM)
3. Deep learning models (Neural Networks, LSTM)
4. Hybrid ensemble approaches

In [ ]:
# 4.1 Traditional Time Series Models
print("📈 Training traditional time series models...")
print("   🔄 ARIMA/SARIMA models...")
arima_models = predictor.fit_arima_models(y_train, seasonal=True)

print("   🔄 Exponential Smoothing...")
ets_models = predictor.fit_exponential_smoothing(y_train)

print(f"   ✅ Fitted {len(arima_models) + len(ets_models)} time series models")

In [ ]:
# 4.2 Machine Learning Models
print("🤖 Training machine learning models...")
print("   🔄 Random Forest, XGBoost, LightGBM...")
ml_models = predictor.fit_machine_learning_models(X_train, y_train, X_val, y_val)

print(f"   ✅ Fitted {len(ml_models)} machine learning models")

# Display feature importance for one model
if 'RandomForest' in predictor.feature_importance:
    print("\n🔍 Top 10 Most Important Features (Random Forest):")
    rf_importance = predictor.feature_importance['RandomForest']
    top_features = sorted(rf_importance.items(), key=lambda x: x[1], reverse=True)[:10]
    for i, (feature, importance) in enumerate(top_features, 1):
        print(f"   {i:2d}. {feature:<30} {importance:.4f}")

In [ ]:
# 4.3 Deep Learning Models
print("🧠 Training deep learning models...")
print("   🔄 Neural Networks and LSTM...")
dl_models = predictor.fit_deep_learning_models(X_train, y_train, X_val, y_val)

print(f"   ✅ Fitted {len(dl_models)} deep learning models")

In [ ]:
# 4.4 Hybrid Ensemble Model
print("🔀 Creating hybrid ensemble model...")
ensemble_models = predictor.create_hybrid_ensemble(X_train, y_train, ['RandomForest', 'XGBoost', 'LightGBM'])

print(f"   ✅ Created ensemble with {len(predictor.models['Ensemble']['base_models'])} base models")

print(f"\n🎯 Total models trained: {len(predictor.models)}")
print("   Models:", list(predictor.models.keys()))

## 5. Model Evaluation and Comparison

We'll evaluate all models using multiple metrics relevant to water demand forecasting.

In [ ]:
# Evaluate all models
print("📊 Evaluating models on test set...")
results = predictor.evaluate_models(X_test, y_test)

print(f"✅ Evaluated {len(results)} models")

In [ ]:
# Comprehensive model comparison
comparison_df = predictor.compare_models()

# Create a more detailed visualization
if not comparison_df.empty:
    # Performance metrics visualization
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=['Mean Absolute Error (MAE)', 'Root Mean Square Error (RMSE)', 
                       'Mean Absolute Percentage Error (MAPE)', 'R² Score'],
        specs=[[{}, {}], [{}, {}]]
    )
    
    metrics = ['MAE', 'RMSE', 'MAPE', 'R2']
    positions = [(1,1), (1,2), (2,1), (2,2)]
    
    for metric, (row, col) in zip(metrics, positions):
        fig.add_trace(
            go.Bar(
                x=comparison_df.index,
                y=comparison_df[metric],
                name=metric,
                showlegend=False
            ),
            row=row, col=col
        )
    
    fig.update_layout(height=600, title_text="Model Performance Comparison")
    fig.show()
    
    # Best models summary
    print("\n🏆 BEST PERFORMING MODELS:")
    print("=" * 50)
    
    best_mae = comparison_df['MAE'].idxmin()
    best_rmse = comparison_df['RMSE'].idxmin()
    best_mape = comparison_df['MAPE'].idxmin()
    best_r2 = comparison_df['R2'].idxmax()
    best_peak = comparison_df['Peak_MAE'].idxmin()
    best_direction = comparison_df['Directional_Accuracy'].idxmax()
    
    print(f"🎯 Overall Accuracy (MAE): {best_mae} ({comparison_df.loc[best_mae, 'MAE']:.3f})")
    print(f"📐 Precision (RMSE): {best_rmse} ({comparison_df.loc[best_rmse, 'RMSE']:.3f})")
    print(f"📊 Percentage Error (MAPE): {best_mape} ({comparison_df.loc[best_mape, 'MAPE']:.2f}%)")
    print(f"🔗 Explained Variance (R²): {best_r2} ({comparison_df.loc[best_r2, 'R2']:.3f})")
    print(f"⚡ Peak Demand Accuracy: {best_peak} ({comparison_df.loc[best_peak, 'Peak_MAE']:.3f})")
    print(f"📈 Directional Accuracy: {best_direction} ({comparison_df.loc[best_direction, 'Directional_Accuracy']:.1f}%)")

## 6. Feature Importance Analysis

Understanding which features drive water demand predictions in Ohrid.

In [ ]:
# Feature importance visualization
predictor.plot_feature_importance(top_n=15)

# Create a combined feature importance analysis
if len(predictor.feature_importance) > 0:
    print("\n🔍 FEATURE IMPORTANCE INSIGHTS:")
    print("=" * 50)
    
    # Combine importance scores from all models
    all_features = set()
    for model_importance in predictor.feature_importance.values():
        all_features.update(model_importance.keys())
    
    # Calculate average importance across models
    avg_importance = {}
    for feature in all_features:
        importances = []
        for model_importance in predictor.feature_importance.values():
            if feature in model_importance:
                importances.append(model_importance[feature])
        if importances:
            avg_importance[feature] = np.mean(importances)
    
    # Top features
    top_avg_features = sorted(avg_importance.items(), key=lambda x: x[1], reverse=True)[:10]
    
    print("Top 10 Most Important Features (Average across models):")
    for i, (feature, importance) in enumerate(top_avg_features, 1):
        print(f"   {i:2d}. {feature:<35} {importance:.4f}")
    
    # Categorize features
    feature_categories = {
        'Temporal': ['hour', 'day_of_week', 'month', 'is_weekend', 'is_holiday', 'hour_sin', 'hour_cos', 'day_of_week_sin', 'day_of_week_cos', 'month_sin', 'month_cos'],
        'Weather': ['temperature', 'humidity', 'precipitation', 'wind_speed', 'pressure', 'cloud_cover'],
        'Tourism': ['tourists_estimated', 'is_tourist_season', 'is_festival_period'],
        'Historical': [f for f in all_features if 'lag_' in f or 'rolling_' in f],
        'Interaction': [f for f in all_features if 'interaction' in f]
    }
    
    print("\n📊 Feature Category Importance:")
    for category, features in feature_categories.items():
        category_importance = sum(avg_importance.get(f, 0) for f in features if f in avg_importance)
        print(f"   {category:<15}: {category_importance:.4f}")

## 7. Prediction Visualization

Visual analysis of model predictions vs actual demand.

In [ ]:
# Predictions visualization
print("📈 Generating prediction visualizations...")

# Select best performing models for visualization
if not comparison_df.empty:
    # Get top 4 models by MAE
    top_models = comparison_df.nsmallest(4, 'MAE').index.tolist()
    predictor.plot_predictions(y_test, model_names=top_models, days_to_show=14)
    
    print(f"📊 Showing predictions for top 4 models: {', '.join(top_models)}")

In [ ]:
# Residual analysis for best model
if not comparison_df.empty:
    best_model = comparison_df['MAE'].idxmin()
    best_predictions = predictor.evaluation_results[best_model]['predictions']
    residuals = y_test.values - best_predictions
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(f'Residual Analysis - {best_model} (Best Model)', fontsize=16)
    
    # Residuals over time
    axes[0,0].plot(residuals[:168])  # First week
    axes[0,0].set_title('Residuals Over Time (First Week)')
    axes[0,0].set_xlabel('Hours')
    axes[0,0].set_ylabel('Residual (m³/hour)')
    axes[0,0].grid(True, alpha=0.3)
    
    # Residual distribution
    axes[0,1].hist(residuals, bins=50, alpha=0.7, edgecolor='black')
    axes[0,1].set_title('Residual Distribution')
    axes[0,1].set_xlabel('Residual (m³/hour)')
    axes[0,1].set_ylabel('Frequency')
    axes[0,1].grid(True, alpha=0.3)
    
    # Predicted vs Actual
    axes[1,0].scatter(y_test.values, best_predictions, alpha=0.5, s=1)
    axes[1,0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
    axes[1,0].set_title('Predicted vs Actual')
    axes[1,0].set_xlabel('Actual (m³/hour)')
    axes[1,0].set_ylabel('Predicted (m³/hour)')
    axes[1,0].grid(True, alpha=0.3)
    
    # Q-Q plot for normality check
    from scipy import stats
    stats.probplot(residuals, dist="norm", plot=axes[1,1])
    axes[1,1].set_title('Q-Q Plot (Normality Check)')
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📊 Residual Analysis for {best_model}:")
    print(f"   Mean residual: {residuals.mean():.4f} m³/hour")
    print(f"   Std residual: {residuals.std():.4f} m³/hour")
    print(f"   Max absolute residual: {abs(residuals).max():.4f} m³/hour")
    
    # Shapiro-Wilk test for normality (on a sample)
    sample_residuals = np.random.choice(residuals, min(5000, len(residuals)), replace=False)
    shapiro_stat, shapiro_p = stats.shapiro(sample_residuals)
    print(f"   Normality test (Shapiro-Wilk): p-value = {shapiro_p:.4f}")
    print(f"   Residuals {'appear normal' if shapiro_p > 0.05 else 'deviate from normality'}")

## 8. Peak Demand Analysis

Special focus on peak demand periods which are critical for infrastructure planning.

In [ ]:
# Peak demand analysis
print("⚡ Peak Demand Analysis")
print("=" * 50)

# Define peak periods (top 10% of demand)
peak_threshold = y_test.quantile(0.9)
peak_mask = y_test >= peak_threshold
peak_periods = y_test[peak_mask]

print(f"Peak demand threshold: {peak_threshold:.2f} m³/hour")
print(f"Number of peak demand hours: {peak_mask.sum()} ({peak_mask.mean()*100:.1f}% of test period)")

if not comparison_df.empty:
    # Peak demand performance comparison
    peak_performance = {}
    
    for model_name in predictor.evaluation_results.keys():
        predictions = predictor.evaluation_results[model_name]['predictions']
        peak_predictions = predictions[peak_mask]
        
        if len(peak_predictions) > 0:
            peak_mae = mean_absolute_error(peak_periods, peak_predictions)
            peak_mape = np.mean(np.abs((peak_periods - peak_predictions) / peak_periods)) * 100
            
            peak_performance[model_name] = {
                'Peak_MAE': peak_mae,
                'Peak_MAPE': peak_mape
            }
    
    peak_df = pd.DataFrame(peak_performance).T
    peak_df = peak_df.sort_values('Peak_MAE')
    
    print("\n🏆 Peak Demand Performance Ranking:")
    print(peak_df.round(3))
    
    best_peak_model = peak_df.index[0]
    print(f"\n⭐ Best model for peak demand: {best_peak_model}")
    print(f"   Peak MAE: {peak_df.loc[best_peak_model, 'Peak_MAE']:.3f} m³/hour")
    print(f"   Peak MAPE: {peak_df.loc[best_peak_model, 'Peak_MAPE']:.2f}%")

## 9. Tourism Impact Assessment

Analysis of how well models capture tourism-driven demand patterns.

In [ ]:
# Tourism impact on model performance
print("🏖️ Tourism Impact Assessment")
print("=" * 50)

# Get test data with tourism information
test_data = df.iloc[-len(X_test):].copy()
test_data['actual_demand'] = y_test.values

# Add predictions from best model
if not comparison_df.empty:
    best_model = comparison_df['MAE'].idxmin()
    test_data['predicted_demand'] = predictor.evaluation_results[best_model]['predictions']
    test_data['prediction_error'] = abs(test_data['actual_demand'] - test_data['predicted_demand'])
    
    # Performance during different tourism periods
    tourism_performance = test_data.groupby('is_tourist_season').agg({
        'prediction_error': ['mean', 'std'],
        'actual_demand': 'mean',
        'tourists_estimated': 'mean'
    }).round(3)
    
    print("Model Performance by Tourism Season:")
    print(tourism_performance)
    
    # Festival periods performance
    festival_performance = test_data.groupby('is_festival_period').agg({
        'prediction_error': ['mean', 'std'],
        'actual_demand': 'mean'
    }).round(3)
    
    print("\nModel Performance during Festival Periods:")
    print(festival_performance)
    
    # Visualization
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Tourism season comparison
    tourism_data = []
    for season in [True, False]:
        season_data = test_data[test_data['is_tourist_season'] == season]
        tourism_data.append(season_data['prediction_error'])
    
    axes[0].boxplot(tourism_data, labels=['Tourist Season', 'Off Season'])
    axes[0].set_title('Prediction Error by Tourism Season')
    axes[0].set_ylabel('Absolute Error (m³/hour)')
    axes[0].grid(True, alpha=0.3)
    
    # Monthly performance
    monthly_error = test_data.groupby('month')['prediction_error'].mean()
    axes[1].bar(monthly_error.index, monthly_error.values)
    axes[1].set_title('Monthly Prediction Error')
    axes[1].set_xlabel('Month')
    axes[1].set_ylabel('Mean Absolute Error (m³/hour)')
    axes[1].grid(True, alpha=0.3)
    
    # Highlight tourist season months
    tourist_months = [6, 7, 8, 5, 9]  # Peak and shoulder seasons
    for month in tourist_months:
        if month in monthly_error.index:
            axes[1].patches[month-1].set_color('orange')
    
    plt.tight_layout()
    plt.show()
    
    # Key insights
    tourist_error = test_data[test_data['is_tourist_season']]['prediction_error'].mean()
    off_season_error = test_data[~test_data['is_tourist_season']]['prediction_error'].mean()
    
    print(f"\n📊 Key Tourism Impact Insights:")
    print(f"   Average error during tourist season: {tourist_error:.3f} m³/hour")
    print(f"   Average error during off season: {off_season_error:.3f} m³/hour")
    
    if tourist_error > off_season_error:
        print(f"   ⚠️ Model struggles more during tourist season (+{((tourist_error/off_season_error-1)*100):.1f}% higher error)")
    else:
        print(f"   ✅ Model performs better during tourist season (-{((1-tourist_error/off_season_error)*100):.1f}% lower error)")

## 10. Model Deployment Recommendations

Summary and recommendations for production deployment.

In [ ]:
# Save models
print("💾 Saving trained models...")
predictor.save_models("../models/")

# Generate deployment recommendations
print("\n" + "="*80)
print("🚀 MODEL DEPLOYMENT RECOMMENDATIONS FOR OHRID WATER UTILITY")
print("="*80)

if not comparison_df.empty:
    # Overall best model
    best_overall = comparison_df['MAE'].idxmin()
    best_peak = comparison_df['Peak_MAE'].idxmin()
    best_direction = comparison_df['Directional_Accuracy'].idxmax()
    
    print(f"\n🏆 RECOMMENDED PRIMARY MODEL: {best_overall}")
    print(f"   • Best overall accuracy (MAE: {comparison_df.loc[best_overall, 'MAE']:.3f} m³/hour)")
    print(f"   • MAPE: {comparison_df.loc[best_overall, 'MAPE']:.2f}%")
    print(f"   • R²: {comparison_df.loc[best_overall, 'R2']:.3f}")
    
    print(f"\n⚡ RECOMMENDED FOR PEAK DEMAND: {best_peak}")
    print(f"   • Best peak demand accuracy (Peak MAE: {comparison_df.loc[best_peak, 'Peak_MAE']:.3f} m³/hour)")
    
    print(f"\n📈 RECOMMENDED FOR TREND ANALYSIS: {best_direction}")
    print(f"   • Best directional accuracy ({comparison_df.loc[best_direction, 'Directional_Accuracy']:.1f}%)")
    
    print("\n🎯 DEPLOYMENT STRATEGY:")
    print("   1. Primary forecasting: Use", best_overall, "for daily operational planning")
    print("   2. Peak demand alerts: Use", best_peak, "for infrastructure stress monitoring")
    print("   3. Ensemble approach: Combine top 3 models for critical decisions")
    
    print("\n📊 MONITORING RECOMMENDATIONS:")
    print("   • Update models monthly with new data")
    print("   • Monitor tourism data for seasonal adjustments")
    print("   • Track prediction errors during festival periods")
    print("   • Implement automated alerts for prediction deviations >20%")
    
    print("\n🔧 INFRASTRUCTURE CONSIDERATIONS:")
    print("   • Peak demand preparation during July-August (tourist season)")
    print("   • Network maintenance scheduling during low-demand periods (Nov-Mar)")
    print("   • Emergency capacity planning for festival periods (Ohrid Summer Festival)")
    
    # Feature-based recommendations
    if predictor.feature_importance:
        print("\n📈 DATA COLLECTION PRIORITIES:")
        all_importance = {}
        for model_importance in predictor.feature_importance.values():
            for feature, importance in model_importance.items():
                all_importance[feature] = all_importance.get(feature, 0) + importance
        
        top_features = sorted(all_importance.items(), key=lambda x: x[1], reverse=True)[:5]
        print("   High priority data sources:")
        for i, (feature, _) in enumerate(top_features, 1):
            if 'tourist' in feature.lower():
                print(f"   {i}. Tourism data (booking platforms, hotel occupancy)")
            elif 'temp' in feature.lower():
                print(f"   {i}. Weather stations (temperature monitoring)")
            elif 'lag' in feature.lower():
                print(f"   {i}. Historical demand data (automated meter readings)")
            elif 'hour' in feature.lower() or 'day' in feature.lower():
                print(f"   {i}. Temporal patterns (calendar integration)")
            else:
                print(f"   {i}. {feature} data")

print("\n✅ RESEARCH FRAMEWORK DEMONSTRATION COMPLETE!")
print("\n📚 Next Steps for Real Implementation:")
print("   1. Collect real water demand data from Ohrid utility")
print("   2. Integrate live weather API feeds")
print("   3. Connect to tourism data sources")
print("   4. Deploy models to GCP for automated forecasting")
print("   5. Build dashboard for utility operators")
print("   6. Implement continuous model retraining pipeline")

## Conclusion

This notebook has demonstrated a comprehensive water demand prediction framework specifically tailored for Ohrid, North Macedonia. The framework successfully:

### Key Achievements:
1. **Generated realistic synthetic data** capturing Ohrid's unique characteristics
2. **Implemented multiple modeling approaches** from traditional time series to deep learning
3. **Evaluated models comprehensively** with metrics relevant to water utilities
4. **Analyzed tourism impact** on demand patterns and model performance
5. **Provided actionable recommendations** for deployment

### Research Contributions:
- **Regional Adaptation**: Framework specifically designed for Balkan/Mediterranean tourism-dependent cities
- **Multi-modal Approach**: Combines traditional and modern forecasting techniques
- **Tourism Integration**: Explicit modeling of UNESCO World Heritage site tourism patterns
- **Peak Demand Focus**: Special attention to infrastructure-critical peak periods
- **Practical Deployment**: Cloud-ready framework with GCP integration

### Next Steps:
The framework is ready for:
1. **Real Data Integration**: Replace synthetic data with actual utility data
2. **Production Deployment**: Deploy to GCP for operational use
3. **Continuous Learning**: Implement automated model retraining
4. **Dashboard Development**: Create operational interfaces for utility staff
5. **Research Publication**: Document methodology and results for academic contribution

This research provides a solid foundation for water demand prediction in tourism-dependent cities and can be adapted for similar locations worldwide.